## Setup

In [46]:
#build out the loop
import random
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np
from bs4 import BeautifulSoup
from requests import get
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from functools import reduce

C:\Users\Daniel\AppData\Local\Temp\ipykernel_5620\853353486.py:10: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [2]:
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.add_argument('--headless')

In [3]:
exe_path = r"C:\bin\geckodriver-v0.32.0-win64\geckodriver.exe"

user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
] 
#proxies = [{'https': 'https://45.8.179.242'}, {'https': 'https://134.209.110.45'},  {'https': 'https:/110.34.3.229'}, {'https': 'https://47.243.121.74'}] 
#proxies = ['108.170.12.14', '44.204.222.225', '143.198.228.250'] # 108.170.12.14
proxies = {'http': 'http://103.242.119.88'}

In [4]:
user_agent = random.choice(user_agents)
#proxy = random.choice(proxies)
headers = {'User-Agent': user_agent}
user_agent = random.choice(user_agents)

options.add_argument(f"--proxy-server=http://'103.242.119.88':80")



## Scrape without Keyword Search (All apartments)

In [5]:
num_pages = (8786/120)
num_pages = round(num_pages)
pages = np.arange(0, num_pages, 1)

In [6]:
iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []


for page in pages:
    print("page is:", page)
    driver = webdriver.Firefox(options=options)
    driver.get(f'https://vancouver.craigslist.org/search/apa#search=1~gallery~{page}~0')

   

    sleep(randint(3,9))

    html = driver.page_source
    driver.quit()
    html_soup = BeautifulSoup(html, 'html.parser') 

    #define the posts
    posts = html_soup.find_all('li', class_= 'cl-search-result cl-search-view-mode-gallery')

    #extract data item-wise
    for post in posts:

        for a in post.find_all('a', href=True):
            url = a['href']
        post_links.append(url)
        
    iterations += 1

    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

page is: 0
Page 1 scraped successfully!
page is: 1
Page 2 scraped successfully!
page is: 2
Page 3 scraped successfully!
page is: 3
Page 4 scraped successfully!
page is: 4
Page 5 scraped successfully!
page is: 5
Page 6 scraped successfully!
page is: 6
Page 7 scraped successfully!
page is: 7
Page 8 scraped successfully!
page is: 8
Page 9 scraped successfully!
page is: 9
Page 10 scraped successfully!
page is: 10
Page 11 scraped successfully!
page is: 11
Page 12 scraped successfully!
page is: 12
Page 13 scraped successfully!
page is: 13
Page 14 scraped successfully!
page is: 14
Page 15 scraped successfully!
page is: 15
Page 16 scraped successfully!
page is: 16
Page 17 scraped successfully!
page is: 17
Page 18 scraped successfully!
page is: 18
Page 19 scraped successfully!
page is: 19
Page 20 scraped successfully!
page is: 20
Page 21 scraped successfully!
page is: 21
Page 22 scraped successfully!
page is: 22
Page 23 scraped successfully!
page is: 23
Page 24 scraped successfully!
page is: 24

In [7]:
van_no_sauna = pd.DataFrame({'URL': post_links})
print(vans_no_sauna.info())
van_no_sauna.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     8760 non-null   object
dtypes: object(1)
memory usage: 68.6+ KB
None


,URL
0,https://vancouver.craigslist.org/pml/apa/d/map...
1,https://vancouver.craigslist.org/van/apa/d/van...
2,https://vancouver.craigslist.org/nvn/apa/d/nor...
3,https://vancouver.craigslist.org/rds/apa/d/sur...
4,https://vancouver.craigslist.org/nvn/apa/d/nor...
5,https://vancouver.craigslist.org/bnc/apa/d/bur...
6,https://vancouver.craigslist.org/bnc/apa/d/bur...
7,https://vancouver.craigslist.org/van/apa/d/van...
8,https://vancouver.craigslist.org/bnc/apa/d/bur...
9,https://vancouver.craigslist.org/bnc/apa/d/bur...


In [8]:
len(set(van_apts_no_sauna['URL']))

8629

In [9]:
van_no_sauna.to_csv('van_no_sauna.csv', index = False)

## Scrape with Sauna Keyword Search

In [15]:
# get number results manually and divide by 120(results per page)
num_pages = (748/120)
num_pages = round(num_pages)
pages = np.arange(0, num_pages, 1)

In [11]:
iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []


for page in pages:
    print("page is:", page)
    driver = webdriver.Firefox(options=options)
    driver.get(f'https://vancouver.craigslist.org/search/apa?query=sauna#search=1~gallery~{page}~0')

   

    sleep(randint(3,9))

    html = driver.page_source
    driver.quit()
    html_soup = BeautifulSoup(html, 'html.parser') 

    #define the posts
    posts = html_soup.find_all('li', class_= 'cl-search-result cl-search-view-mode-gallery')

    #extract data item-wise
    for post in posts:

        for a in post.find_all('a', href=True):
            url = a['href']
        post_links.append(url)
        
    iterations += 1

    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

page is: 0
Page 1 scraped successfully!
page is: 1
Page 2 scraped successfully!
page is: 2
Page 3 scraped successfully!
page is: 3
Page 4 scraped successfully!
page is: 4
Page 5 scraped successfully!
page is: 5
Page 6 scraped successfully!


Scrape complete!


In [12]:
van_sauna = pd.DataFrame({'URL': post_links})
print(van_sauna.info())
van_sauna.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     720 non-null    object
dtypes: object(1)
memory usage: 5.8+ KB
None


,URL
0,https://vancouver.craigslist.org/van/apa/d/van...
1,https://vancouver.craigslist.org/pml/apa/d/por...
2,https://vancouver.craigslist.org/van/apa/d/van...
3,https://vancouver.craigslist.org/bnc/apa/d/bur...
4,https://vancouver.craigslist.org/van/apa/d/van...
5,https://vancouver.craigslist.org/bnc/apa/d/bur...
6,https://vancouver.craigslist.org/van/apa/d/van...
7,https://vancouver.craigslist.org/van/apa/d/van...
8,https://vancouver.craigslist.org/van/apa/d/van...
9,https://vancouver.craigslist.org/bnc/apa/d/coq...


In [13]:
len(set(van_sauna['URL']))

600

In [14]:
van_sauna.to_csv('van_sauna.csv', index = False)

## Scrape with Pool Keyword Search

In [16]:
num_pages = (1353/120)
num_pages = round(num_pages)
pages = np.arange(0, num_pages, 1)

In [17]:
iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []


for page in pages:
    print("page is:", page)
    driver = webdriver.Firefox(options=options)
    driver.get(f'https://vancouver.craigslist.org/search/apa?query=pool#search=1~gallery~{page}~0')

   

    sleep(randint(3,9))

    html = driver.page_source
    driver.quit()
    html_soup = BeautifulSoup(html, 'html.parser') 

    #define the posts
    posts = html_soup.find_all('li', class_= 'cl-search-result cl-search-view-mode-gallery')

    #extract data item-wise
    for post in posts:

        for a in post.find_all('a', href=True):
            url = a['href']
        post_links.append(url)
        
    iterations += 1

    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

page is: 0
Page 1 scraped successfully!
page is: 1
Page 2 scraped successfully!
page is: 2
Page 3 scraped successfully!
page is: 3
Page 4 scraped successfully!
page is: 4
Page 5 scraped successfully!
page is: 5
Page 6 scraped successfully!
page is: 6
Page 7 scraped successfully!
page is: 7
Page 8 scraped successfully!
page is: 8
Page 9 scraped successfully!
page is: 9
Page 10 scraped successfully!
page is: 10
Page 11 scraped successfully!


Scrape complete!


In [18]:
van_pool = pd.DataFrame({'URL': post_links})
print(van_pool.info())
van_pool.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     1320 non-null   object
dtypes: object(1)
memory usage: 10.4+ KB
None


,URL
0,https://vancouver.craigslist.org/rds/apa/d/sur...
1,https://vancouver.craigslist.org/nvn/apa/d/wes...
2,https://vancouver.craigslist.org/bnc/apa/d/bur...
3,https://vancouver.craigslist.org/nvn/apa/d/nor...
4,https://vancouver.craigslist.org/nvn/apa/d/van...
5,https://vancouver.craigslist.org/rds/apa/d/del...
6,https://vancouver.craigslist.org/van/apa/d/van...
7,https://vancouver.craigslist.org/pml/apa/d/coq...
8,https://vancouver.craigslist.org/van/apa/d/van...
9,https://vancouver.craigslist.org/nvn/apa/d/wes...


In [19]:
len(set(van_pool['URL']))

1200

In [20]:
van_pool.to_csv('van_pool.csv', index = False)

## Scrape with Steam Room Keyword Search

In [21]:
num_pages = (423/120)
num_pages = round(num_pages)
pages = np.arange(0, num_pages, 1)

In [22]:
iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []


for page in pages:
    print("page is:", page)
    driver = webdriver.Firefox(options=options)
    driver.get(f'https://vancouver.craigslist.org/search/apa?query=steam%20room#search=1~gallery~{page}~0')

   

    sleep(randint(3,9))

    html = driver.page_source
    driver.quit()
    html_soup = BeautifulSoup(html, 'html.parser') 

    #define the posts
    posts = html_soup.find_all('li', class_= 'cl-search-result cl-search-view-mode-gallery')

    #extract data item-wise
    for post in posts:

        for a in post.find_all('a', href=True):
            url = a['href']
        post_links.append(url)
        
    iterations += 1

    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

page is: 0
Page 1 scraped successfully!
page is: 1
Page 2 scraped successfully!
page is: 2
Page 3 scraped successfully!
page is: 3
Page 4 scraped successfully!


Scrape complete!


In [23]:
van_steam = pd.DataFrame({'URL': post_links})
print(van_steam.info())
van_steam.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     480 non-null    object
dtypes: object(1)
memory usage: 3.9+ KB
None


,URL
0,https://vancouver.craigslist.org/pml/apa/d/por...
1,https://vancouver.craigslist.org/van/apa/d/van...
2,https://vancouver.craigslist.org/rch/apa/d/ric...
3,https://vancouver.craigslist.org/rch/apa/d/ric...
4,https://vancouver.craigslist.org/rch/apa/d/ric...
5,https://vancouver.craigslist.org/van/apa/d/van...
6,https://vancouver.craigslist.org/bnc/apa/d/coq...
7,https://vancouver.craigslist.org/van/apa/d/van...
8,https://vancouver.craigslist.org/rds/apa/d/sur...
9,https://vancouver.craigslist.org/bnc/apa/d/new...


In [24]:
len(set(van_steam['URL']))

360

In [25]:
van_steam.to_csv('van_steam.csv', index = False)

## Merge No Sauna with Sauna with Pool with Steam Room

In [59]:
van_no_sauna = pd.read_csv('van_no_sauna.csv', parse_dates=True)
van_sauna = pd.read_csv('van_sauna.csv', parse_dates=True)
van_pool = pd.read_csv('van_pool.csv', parse_dates=True)
van_steam = pd.read_csv('van_steam.csv', parse_dates=True)

In [60]:
van_no_sauna['sauna'] = False
van_sauna['sauna'] = True
van_pool['pool'] = True
van_steam['steam'] = True

van_sauna_or_no_sauna = pd.concat([van_sauna, van_no_sauna], ignore_index=True)
van_sauna_or_no_sauna = van_sauna_or_no_sauna.drop_duplicates(subset = 'URL')

In [61]:
van_sauna_or_no_sauna = van_sauna_or_no_sauna.reset_index(drop = True)

In [62]:
van_sauna_or_no_sauna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8643 entries, 0 to 8642
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     8643 non-null   object
 1   sauna   8643 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 76.1+ KB


In [63]:
data_frames = [van_sauna_or_no_sauna, van_pool, van_steam]

In [64]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['URL'],
                                            how='outer'), data_frames)

In [68]:
df_merged

,URL,sauna,pool,steam
0,https://vancouver.craigslist.org/van/apa/d/van...,True,True,True
1,https://vancouver.craigslist.org/van/apa/d/van...,True,True,True
2,https://vancouver.craigslist.org/pml/apa/d/por...,True,NaN,NaN
3,https://vancouver.craigslist.org/van/apa/d/van...,True,NaN,NaN
4,https://vancouver.craigslist.org/bnc/apa/d/bur...,True,NaN,NaN
...,...,...,...,...
8899,https://vancouver.craigslist.org/bnc/apa/d/bur...,NaN,True,NaN
8900,https://vancouver.craigslist.org/van/apa/d/van...,NaN,True,NaN
8901,https://vancouver.craigslist.org/van/apa/d/van...,NaN,True,NaN
8902,https://vancouver.craigslist.org/van/apa/d/van...,NaN,NaN,True


In [66]:
van_sauna_pool_steam = df_merged.drop_duplicates(subset = 'URL')

In [ ]:
for url in van_sauna_pool_steam:
    

In [71]:
van_sauna_pool_steam.to_csv('van_sauna_pool_steam.csv')